In [2]:
import numpy as np
from cython_weave import *
import tensorflow as tf
#from fancy_conv import array_weave_forward

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def create_I_matrix(N, weave_param):
    N_arr = np.arange(1,N**2 + 1).reshape([N,N]).astype(float)
    N_weave,_ = array_weave_fast_forward(N_arr, weave_param)
    N_weave[np.where(N_weave == 0)] = -1
    num_rows, num_cols = N_weave.shape
    rows = np.zeros([num_rows,num_cols])
    cols = np.zeros([num_rows,num_cols])
    for i in range(num_cols):
        for j in range(num_rows):
            i_j_loc = np.where(N_weave[i,j] == N_arr)
            if len(i_j_loc[0]) > 0:
                rows[i,j] = int(i_j_loc[0])
                cols[i,j] = int(i_j_loc[1])
            else:
                rows[i,j] = 0

                cols[i,j] = N
    return (rows.astype(int),cols.astype(int))

def create_full_I_matrix(N, num_images, num_filters, weave_param):
    """
    Creates an matrix of size (num_images, num_filters, height, width, input_tensor_dimension (4))
    
    """
    tensor_dimension = 4
    N_arr = np.arange(1,N**2 * num_filters * num_images + 1).reshape([num_images,num_filters,N,N]).astype(float)
    N_weave,_ = array_weave_fast_forward(N_arr, weave_param)
    N_weave[np.where(N_weave == 0)] = -1
    _,_,num_rows, num_cols = N_weave.shape
    print(N_weave.shape)
    
    I_full = np.zeros([num_images,num_filters,num_rows,num_cols,tensor_dimension])
    
    for img in range(num_images):
        print(img)
        for fil in range(num_filters):
            for i in range(num_cols):
                for j in range(num_rows):
                    i_j_loc = np.where(N_weave[img,fil,i,j] == N_arr)
                    if len(i_j_loc[0]) > 0:
                        I_full[img,fil,i,j] = i_j_loc
                    else:
                        I_full[img,fil,i,j] = np.array([0,0,0,N])
    return I_full.astype(int)

In [4]:
A = np.array([[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]]])
print(A)
test = np.where(A == A)

[[[ 1  2  3]
  [ 4  5  6]]

 [[ 7  8  9]
  [10 11 12]]]


In [5]:
weave_param = {'num_zeros':2, 'filter_size':3}
N = 5
num_images = 4
num_filters = 3
print(create_full_I_matrix(N, num_images, num_filters, weave_param))


(4, 3, 13, 13)
0
1
2
3
[[[[[0 0 0 5]
    [0 0 0 3]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 0 1]
    [0 0 0 5]]

   [[0 0 3 0]
    [0 0 3 3]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 3 1]
    [0 0 3 4]]

   [[0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]]

   ...

   [[0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]]

   [[0 0 1 0]
    [0 0 1 3]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 1 1]
    [0 0 1 4]]

   [[0 0 0 5]
    [0 0 4 3]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 4 1]
    [0 0 0 5]]]


  [[[0 0 0 5]
    [0 1 0 3]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 1 0 1]
    [0 0 0 5]]

   [[0 1 3 0]
    [0 1 3 3]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 1 3 1]
    [0 1 3 4]]

   [[0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]]

   ...

   [[0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]]

   [[0 1 1 0]
    [

In [6]:
weave_param = {'num_zeros':2, 'filter_size':3}
I = create_I_matrix(5,weave_param)
print(I)

(array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [3, 3, 0, 3, 3, 0, 3, 0, 3, 3, 0, 3, 3],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
       [4, 4, 0, 4, 4, 0, 4, 0, 4, 4, 0, 4, 4],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 3, 0, 0, 3, 0, 0, 0, 3, 0, 0, 3, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1],
       [0, 4, 0, 0, 4, 0, 0, 0, 4, 0, 0, 4, 0]]), array([[5, 3, 5, 5, 4, 5, 5, 5, 0, 5, 5, 1, 5],
       [0, 3, 5, 1, 4, 5, 2, 5, 0, 3, 5, 1, 4],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       [5, 3, 5, 5, 4, 5, 5, 5, 0, 5, 5, 1, 5],
       [0, 3, 5, 1, 4, 5, 2, 5, 0, 3, 5, 1, 4],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       [5, 3, 5, 5, 4, 5, 5, 5, 0, 5, 5, 1, 5],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

In [28]:
# Input
N = 5
n_fil = 3
n_img = 2

a = tf.constant(np.arange(1,n_fil*n_img*N**2+1).reshape(n_img,n_fil,N,N), tf.float32)
a = tf.concat((a, np.zeros([n_img,n_fil,N,1])), axis = 3)
image_indx = create_full_I_matrix(N, n_img, n_fil, weave_param)
print(image_indx.shape)

# collect_indx = np.stack([rows,cols])
# print('collect_indx', collect_indx.shape)
# filter_indx = np.stack([collect_indx]* n_fil)
# print('filter_indx', filter_indx.shape)
# image_indx = np.stack([filter_indx]*n_img)
# print('image_indx', image_indx.shape)
# image_indx = np.transpose(image_indx, [0,1,3,4,2])
# print('image_indx', image_indx.shape)
x = tf.placeholder('float32', (n_img,n_fil,16, 16))

with tf.Session() as sess:
    print(tf.shape(a))
    r = sess.run(tf.gather_nd(a, image_indx))
print(r)
print(r.shape)

(2, 3, 13, 13)
0
1
(2, 3, 13, 13, 4)
Tensor("Shape_41:0", shape=(4,), dtype=int32)
[[[[  0.   4.   0. ...   0.   2.   0.]
   [ 16.  19.   0. ...   0.  17.  20.]
   [  0.   0.   0. ...   0.   0.   0.]
   ...
   [  0.   0.   0. ...   0.   0.   0.]
   [  6.   9.   0. ...   0.   7.  10.]
   [  0.  24.   0. ...   0.  22.   0.]]

  [[  0.  29.   0. ...   0.  27.   0.]
   [ 41.  44.   0. ...   0.  42.  45.]
   [  0.   0.   0. ...   0.   0.   0.]
   ...
   [  0.   0.   0. ...   0.   0.   0.]
   [ 31.  34.   0. ...   0.  32.  35.]
   [  0.  49.   0. ...   0.  47.   0.]]

  [[  0.  54.   0. ...   0.  52.   0.]
   [ 66.  69.   0. ...   0.  67.  70.]
   [  0.   0.   0. ...   0.   0.   0.]
   ...
   [  0.   0.   0. ...   0.   0.   0.]
   [ 56.  59.   0. ...   0.  57.  60.]
   [  0.  74.   0. ...   0.  72.   0.]]]


 [[[  0.  79.   0. ...   0.  77.   0.]
   [ 91.  94.   0. ...   0.  92.  95.]
   [  0.   0.   0. ...   0.   0.   0.]
   ...
   [  0.   0.   0. ...   0.   0.   0.]
   [ 81.  84.   0. ... 

In [21]:
def create_full_I_zero_weave_matrix(input_shape, num_images, weave_param):
    _,num_filters,num_rows,num_cols = input_shape
    num_zeros = weave_param['num_zeros']
    tensor_dimension = len(input_shape)
    
    if tensor_dimension != 4:
        raise ValueError('Must operate on 4D tensors')
    if num_rows != num_cols:
        raise ValueErrro('Must operate ona square image')
        
    HH = WW = num_rows*(1+num_zeros) - num_zeros
     
    if num_images is not None:
        N_arr = np.arange(1,num_cols**2 * num_filters * num_images + 1).reshape(
            [num_images,num_filters,num_rows,num_cols]).astype(float)
        
        N_zero_weave = np.zeros([num_images,num_filters,HH,WW])
        N_zero_weave[:,:,::3,::3] = N_arr
        I_full = np.zeros([num_images,num_filters,HH,WW,tensor_dimension])

    N_zero_weave[np.where(N_zero_weave == 0)] = -1
    for img in range(num_images):
        for fil in range(num_filters):
            for i in range(HH):
                for j in range(WW):
                    i_j_loc = np.where(N_zero_weave[img,fil,i,j] == N_arr)
                    if len(i_j_loc[0]) > 0:
                        I_full[img,fil,i,j] = i_j_loc
                    else:
                        I_full[img,fil,i,j] = np.array([0,0,0,num_cols])
   
    return I_full.astype(int)

In [30]:
image_indx = create_full_I_zero_weave_matrix((n_img*2, n_fil, N, N),n_img, weave_param)
print(image_indx.shape)

(10, 3, 13, 13, 4)


In [34]:
N = 5
n_fil = 3
n_img = 10

a =  tf.constant(np.arange(1,n_img*n_fil*N**2 + 1).reshape(n_img,n_fil,N,N), tf.float64)
a = tf.concat((a, tf.zeros_like(a)), axis = 3)

with tf.Session() as sess:
    print(tf.shape(a))
    r = sess.run(tf.gather_nd(a, image_indx[:n_img]))
print(r)

Tensor("Shape_45:0", shape=(4,), dtype=int32)
[[[[  1.   0.   0. ...   0.   0.   5.]
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   ...
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   [ 21.   0.   0. ...   0.   0.  25.]]

  [[ 26.   0.   0. ...   0.   0.  30.]
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   ...
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   [ 46.   0.   0. ...   0.   0.  50.]]

  [[ 51.   0.   0. ...   0.   0.  55.]
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   ...
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   [ 71.   0.   0. ...   0.   0.  75.]]]


 [[[ 76.   0.   0. ...   0.   0.  80.]
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   ...
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   [ 96.   0.   0. ..